In [23]:
import numpy as np
import pandas as pd
import itertools
import json

In [6]:
movies = pd.read_csv("./ml-latest-small/movies.csv")
# movies['genres'].str.split('|')
# set(itertools.chain.from_iterable(movies.genres))
# np.unique([*itertools.chain.from_iterable(movies.genres)])
genre = {}
for l in movies['genres'].str.split('|'):
    for g in l:
        if g not in genre:
            genre[g] = 1
        else:
            genre[g] += 1
            
genre
# movies.head()


{'Adventure': 1263,
 'Animation': 611,
 'Children': 664,
 'Comedy': 3756,
 'Fantasy': 779,
 'Romance': 1596,
 'Drama': 4361,
 'Action': 1828,
 'Crime': 1199,
 'Thriller': 1894,
 'Horror': 978,
 'Mystery': 573,
 'Sci-Fi': 980,
 'War': 382,
 'Musical': 334,
 'Documentary': 440,
 'IMAX': 158,
 'Western': 167,
 'Film-Noir': 87,
 '(no genres listed)': 34}

In [13]:
genretoidx = {}
idxtogenre = []
for i, g in enumerate(genre):
    # print(g, i)
    genretoidx[g] = i
    idxtogenre.append(g)
    
# print(genretoidx, idxtogenre)

In [32]:
def toVec(genres):
    vec = [0] * 20
    # print(genres)
    for g in genres.split('|'):
        # print(g)
        vec[genretoidx[g]] += 1
    return vec
    
genredict = {m[0] : toVec(m[1]) for m in movies[['movieId','genres']].to_numpy()}
# print(genredict)
# json.dumps(genredict)
with open('data.json', 'w') as fp:
    json.dump(genredict, fp)

In [35]:
ratings = pd.read_csv("./ml-latest-small/ratings.csv")
# ratings.groupby('userId').transform(lambda x: (x - x.mean()) / x.std())
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [59]:
def fx(rating, movieId):
    # print(genredict[movieId], rating)
    return (rating * np.array(genredict[movieId])).astype(object)
ratings['rating'] = ratings.groupby('userId')['rating'].transform(lambda x: (x - x.mean()) / x.std())
ratings['genrating'] = np.vectorize(fx)(ratings['rating'], ratings['movieId'])
ratings.head()

,userId,movieId,rating,timestamp,genrating
0,1,1,-0.457947,964982703,"[-0.45794663435835375, -0.45794663435835375, -..."
1,1,3,-0.457947,964981247,"[-0.0, -0.0, -0.0, -0.45794663435835375, -0.0,..."
2,1,6,-0.457947,964982224,"[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...."
3,1,47,0.791978,964983815,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1,50,0.791978,964982931,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.791..."


In [1]:
ratings.groupby('userId')['genrating']


NameError: name 'ratings' is not defined